# 🚀 Requête WIKIDATA

## 🔨 Construction de l'environnement nécessaire et configuration

### Installation des modules

In [ ]:
%pip install SPARQLWrapper tqdm pandas

print("⬇️ Installation terminée !")

### Configuration

In [4]:
# 🔧 CONFIGURATION ET IMPORTS
import json
import csv
import time
import re
import os
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import pandas as pd

# Configuration
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
RATE_LIMIT_DELAY = 2.0 
BATCH_SIZE = 10  
MAX_RETRIES = 5  
REQUEST_TIMEOUT = 60
ENRICHMENT_BATCH_SIZE = 15

# Dossier de sortie
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

print("🚀 CONFIGURATION TERMINÉE")
print(f"📁 Dossier de sortie: {output_dir}")
print(f"⏱️  Rate limit: {RATE_LIMIT_DELAY}s entre requêtes")
print(f"📦 Taille des batches: {BATCH_SIZE}")

🚀 CONFIGURATION TERMINÉE
📁 Dossier de sortie: ./output
⏱️  Rate limit: 2.0s entre requêtes
📦 Taille des batches: 10


In [5]:
def create_sparql_client():
    """
    Créer un client SPARQL pour interagir avec Wikidata
    :return: Instance de SPARQLWrapper configurée pour Wikidata
    """
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(REQUEST_TIMEOUT)
    return sparql

def execute_sparql_query(query, max_retries=MAX_RETRIES):
    """
    Exécute une requête SPARQL avec gestion des erreurs et rate limiting
    :param query: La requête SPARQL à exécuter
    :param max_retries: Nombre maximum de tentatives en cas d'échec
    :return: Résultats de la requête ou une liste vide en cas d'échec
    """
    sparql = create_sparql_client()
    for attempt in range(max_retries):
        try:
            sparql.setQuery(query)  # Définition de la requête SPARQL à exécuter
            results = sparql.query().convert()  # Exécution de a requête et conversion en JSON
            time.sleep(RATE_LIMIT_DELAY)  # Délai entre les requêtes
            return results["results"]["bindings"]  # Résultats extraits
        except Exception as e:
            print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée: {e}...")  # Affiche l'erreur et le numéro de tentative
            if attempt < max_retries - 1:
                time.sleep(RATE_LIMIT_DELAY * (attempt + 2))  # Attend plus longtemps avant de réessayer si les tentatives max ne sont pas dépassées
            else:
                print(f"❌ Requête échouée après {max_retries} tentatives") 
                return []
    return []

def clean_entity_id(entity_uri):
    """
    Extrait l'ID d'une entité à partir de son URI
    :param entity_uri: URI de l'entité (ex: "http://www.wikidata.org/entity/Q42'")
    :return: ID de l'entité (ex: "Q42") ou une chaîne vide si l'URI est vide
    """
    if not entity_uri:
        return ""
    return entity_uri.split("/")[-1] if "/" in entity_uri else entity_uri

def execute_batch_queries(queries, description="Requêtes"):
    """
    Exécute une liste de requêtes SPARQL en batch
    :param queries: Liste de requêtes SPARQL à exécuter
    :param description: Description de la tâche pour l'affichage
    :return: Liste de tous les résultats combinés
    """
    all_results = []
    for i, query in enumerate(tqdm(queries, desc=description)):  # Boucle sur chaque requête avec barre de progression
        results = execute_sparql_query(query)  # Exécute la requête SPARQL
        all_results.extend(results)  # Ajoute les résultats à la liste globale
        if (i + 1) % BATCH_SIZE == 0:  # Toutes les BATCH_SIZE requêtes, on attend un peu
            time.sleep(RATE_LIMIT_DELAY)  # Pause
    return all_results  # Retourne la liste complète des résultats

print("🔗 FONCTIONS CONFIGURÉES")

🔗 FONCTIONS CONFIGURÉES AVEC DIAGNOSTIC AVANCÉ


## 🚧 Construction de la requête


### Rechercher une entité par nom

In [ ]:
# 🎯 IDENTIFICATION DE L'ENTITÉ

def find_aeronautics_entity():
  """
  Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
  :return: Tuple contenant l'ID, le label et la description de l'entité
  """
  entity_name = input("Entrez le nom de l'entité à rechercher (ex: aeronautics) : ").strip()
  if not entity_name:
    print("❌ Aucun nom d'entité fourni.")
    return None, None, None

  aeronautics_query = f"""
  SELECT DISTINCT ?item ?itemLabel ?itemDescription WHERE {{
    ?item rdfs:label ?label .
    FILTER(LANG(?label) = "fr" || LANG(?label) = "en") .
    FILTER(REGEX(?label, "{re.escape(entity_name)}", "i"))
    SERVICE wikibase:label {{ 
      bd:serviceParam wikibase:language "fr,en" . 
    }}
  }}
  LIMIT 5
  """

  print(f"🔍 Recherche de l'entité principale '{entity_name}'...")
  results = execute_sparql_query(aeronautics_query)

  if results:
    entity = results[0]
    entity_id = clean_entity_id(entity["item"]["value"])
    entity_label = entity["itemLabel"]["value"]
    entity_desc = entity.get("itemDescription", {}).get("value", "")

    print(f"✅ Entité trouvée: {entity_label} ({entity_id})")
    print(f"📄 Description: {entity_desc}")

    return entity_id, entity_label, entity_desc
  else:
    print("❌ Entité non trouvée")
    return None, None, None

# Identifier l'entité principale
aeronautics_entity, aeronautics_label, aeronautics_desc = find_aeronautics_entity()

### CELLULE DE TEST : 
```sql
SELECT ?item ?itemLabel ?parent 
WHERE {
  ?item wdt:P31 ?parent.
  FILTER(CONTAINS(LCASE(?parent), "aéronautique")).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}

LIMIT 10
```


In [ ]:
# 🎯 IDENTIFICATION DE L'ENTITÉ

def find_aeronautics_entity():
  """
  Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
  :return: Tuple contenant l'ID, le label et la description de l'entité
  """
  entity_name = input("Entrez le nom de l'entité à rechercher (ex: aeronautics) : ").strip()
  if not entity_name:
    print("❌ Aucun nom d'entité fourni.")
    return None, None, None
  
  aeronautics_query = f"""
  SELECT ?item ?itemLabel ?parent ?itemDescription
WHERE {{
  ?item wdt:P31 ?parent.
  ?parent rdfs:label ?parentLabel.
  FILTER(LANG(?parentLabel) = "fr").
  FILTER(CONTAINS(LCASE(?parentLabel), "{entity_name}")).
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr". }}
}}

LIMIT 1
"""

  print(f"🔍 Recherche de l'entité principale '{entity_name}'...")
  results = execute_sparql_query(aeronautics_query)

  if results:
    entity = results[0]
    entity_id = clean_entity_id(entity["item"]["value"])
    entity_label = entity["itemLabel"]["value"]
    entity_desc = entity.get("itemDescription", {}).get("value", "")

    print(f"✅ Entité trouvée: {entity_label} ({entity_id})")
    print(f"📄 Description: {entity_desc}")

    return entity_id, entity_label, entity_desc
  else:
    print("❌ Entité non trouvée")
    return None, None, None

# Identifier l'entité principale
aeronautics_entity, aeronautics_label, aeronautics_desc = find_aeronautics_entity()

🔍 Recherche de l'entité principale 'aéronautique'...
⚠️  Tentative 1/5 échouée: The read operation timed out...
⚠️  Tentative 2/5 échouée: The read operation timed out...
⚠️  Tentative 3/5 échouée: The read operation timed out...


### Requêtes SPARQL

In [ ]:
def build_aeronautics_extraction_queries():
    """Construit les requêtes d'extraction des données"""
    queries = {
        "manufacturers": """
        SELECT DISTINCT ?item 
              
                ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=",") AS ?synonyms_fr)
        WHERE {
          # Tous les équipements d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361*/wdt:P452*/wdt:P749* wd:Q936518 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français (P1709 est "synonymes exacts")
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_models": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les modèles d'avions (instances ou sous-classes ou parties)
          ?item wdt:P31/wdt:P279* wd:Q11436 .
          
        
          # On cherche le parent immédiat selon différentes relations         
            OPTIONAL { ?item wdt:P179 ?series .}
            OPTIONAL { ?item wdt:176 ?constructeur.}
            OPTIONAL { ?item wdt:P31 ?instanceOf . } 
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
                       
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?series, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_components": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les équipements d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361* wd:Q16693356 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aeronautic_profession": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
        ?item wdt:P425* ?domaine.
        VALUES ?domaine { wd:Q765633 wd:Q906438 wd:Q1434048 wd:Q206814 wd:Q627716 wd:Q221395 wd:Q765633 wd:Q22719}.  
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur, ?domaine) AS ?parent) # Attention à coalesce pour éviter les doublons
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """
    }
    return queries

print("✅ Requêtes définies (avec synonymes français)")

## 🔎 Lancer la recherche

In [ ]:
def extract_all_aeronautics_data():
    """Extrait toutes les données aéronautiques de manière optimisée"""
    print("🏗️ EXTRACTION HIÉRARCHIQUE EXHAUSTIVE")
    print("="*50)
    
    queries = build_aeronautics_extraction_queries()
    all_results = []
    
    for category, query in queries.items():
        print(f"\n🔍 Extraction: {category}")
        results = execute_sparql_query(query)
        
        # Enrichir chaque résultat avec sa catégorie
        for result in results:
            result["source_category"] = category
        
        all_results.extend(results)
        print(f"✅ {len(results)} entités trouvées pour {category}")
    
    print(f"\n🎯 TOTAL: {len(all_results)} entités extraites")
    return all_results

raw_aeronautics_data = extract_all_aeronautics_data()

### Aperçu

In [ ]:
print(raw_aeronautics_data[:5])  # pour afficher un aperçu

json_filename = f"raw.json"
json_filepath = os.path.join(output_dir, json_filename)

with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(raw_aeronautics_data, jsonfile, ensure_ascii=False, indent=2)

## 📁 Export

### Construction du fichier

In [ ]:
# 🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE

# def get_id_from_uri(uri):
#     # Ex: "http://www.wikidata.org/entity/Q105557" → "Q105557"
#     return uri.split("/")[-1] if uri else ""

def build_final_hierarchy(raw_aeronautics_data):
    """Construit la hiérarchie finale avec parents immédiats et catégories"""
    print("🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE")
    print("="*45)
    
    # Créer la hiérarchie structurée
    hierarchy = []
    for entry in raw_aeronautics_data:
    
        hierarchy.append(
            {
            "ID": clean_entity_id(entry.get("item", {}).get("value", "")),
            "Terme": entry.get("itemLabel", {}).get("value", ""),
            "ID_TG": clean_entity_id(entry.get("parent", {}).get("value", "")),
            "TG": entry.get("parentLabel", {}).get("value", ""),
            "Def": entry.get("itemDescription", {}).get("value", ""),
            "EP": entry.get("synonyms_fr", {}).get("value", ""),
            "TA": entry.get("source_category", {})
        }
        )
    
 
    print(f"✅ Hiérarchie construite: {len(hierarchy)} entrées totales")
    return hierarchy

final_thesaurus = build_final_hierarchy(raw_aeronautics_data)
print(f"🎯 Thésaurus final: {len(final_thesaurus)} entrées")

### Export du fichier

In [ ]:
# 💾 EXPORT FINAL UNIQUE - CSV Occidental European Format (semicolon separated)

import os
import csv
import json
from datetime import datetime

def export_final_thesaurus(thesaurus_data):
    """Exporte le thésaurus final en CSV (point-virgule, format européen) et JSON"""
    print("💾 EXPORT FINAL UNIQUE")
    print("="*25)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export CSV - Occidental European (semicolon separator, utf-8-sig BOM)
    csv_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.csv"
    csv_filepath = os.path.join(output_dir, csv_filename)
    
    fieldnames = [
        'ID', 'Terme', 'ID_TG','TG', 'Def', 'EP',
        'TA'
    ]
    
    print(f"📄 Export CSV: {csv_filename}")
    with open(csv_filepath, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(
            csvfile, 
            fieldnames=fieldnames,
            delimiter=';',         # Use semicolon as separator
            quoting=csv.QUOTE_MINIMAL
        )
        writer.writeheader()
        for entry in sorted(thesaurus_data, key=lambda x: x["ID"]):
            # Ensure all values are strings and convert None to empty string
            row = {k: ('' if v is None else str(v)) for k, v in entry.items()}
            # Guarantee all required fields exist in row
            for field in fieldnames:
                row.setdefault(field, '')
            writer.writerow(row)
    
    # 2. Export JSON avec métadonnées
    json_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.json"
    json_filepath = os.path.join(output_dir, json_filename)
    
    stats = analyze_thesaurus_statistics(thesaurus_data)
    
    json_data = {
        "metadata": {
            "title": "Thésaurus Aéronautique Final - Wikidata",
            "description": "Thésaurus exhaustif avec hiérarchie et parents immédiats",
            "version": "1.0-FINAL",
            "created": timestamp,
            "source": "Wikidata SPARQL optimisé",
            "total_entries": len(thesaurus_data),
            "extraction_method": "multi-query_hierarchical",
            "parent_detection": "automatic_wikidata_relations",
            "multilingual_support": True,
            "format": "structured_hierarchical_thesaurus"
        },
        "statistics": stats,
        "data": thesaurus_data
    }
    
    print(f"📄 Export JSON: {json_filename}")
    with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
        json.dump(json_data, jsonfile, ensure_ascii=False, indent=2)
    
    return csv_filepath, json_filepath, stats

def analyze_thesaurus_statistics(thesaurus_data):
    """Analyse les statistiques du thésaurus final"""
    stats = {
        "total_entries": len(thesaurus_data),
        "categories": {},
        "relation_types": {},
        "languages": {},
        "hierarchy_depth": 0,
        "entries_with_synonyms": 0,
        "entries_with_descriptions": 0
    }
    
    for entry in thesaurus_data:
        # Catégories
        category = entry.get("category", "unknown")
        stats["categories"][category] = stats["categories"].get(category, 0) + 1
        
        # Types de relation
        rel_type = entry.get("relation_type", "unknown")
        stats["relation_types"][rel_type] = stats["relation_types"].get(rel_type, 0) + 1
        
        # Langues
        lang = entry.get("lang", "unknown")
        stats["languages"][lang] = stats["languages"].get(lang, 0) + 1
        
        # Enrichissements
        if entry.get("synonyms"):
            stats["entries_with_synonyms"] += 1
        if entry.get("description"):
            stats["entries_with_descriptions"] += 1
    
    return stats

def display_final_summary(stats, csv_file, json_file):
    """Affiche un résumé final du thésaurus généré"""
    print("\n🎯 RÉSUMÉ FINAL DU THÉSAURUS AÉRONAUTIQUE")
    print("="*50)
    
    print(f"📊 STATISTIQUES GÉNÉRALES:")
    print(f"   • Total d'entrées: {stats['total_entries']}")
    print(f"   • Entrées avec synonymes: {stats['entries_with_synonyms']}")
    print(f"   • Entrées avec descriptions: {stats['entries_with_descriptions']}")
    
    print(f"\n📂 RÉPARTITION PAR CATÉGORIE:")
    for category, count in sorted(stats["categories"].items(), key=lambda x: x[1], reverse=True):
        percentage = (count / stats['total_entries']) * 100
        print(f"   • {category}: {count} ({percentage:.1f}%)")
    
    print(f"\n🔗 TYPES DE RELATIONS:")
    for rel_type, count in sorted(stats["relation_types"].items(), key=lambda x: x[1], reverse=True):
        print(f"   • {rel_type}: {count}")
    
    print(f"\n🌐 LANGUES:")
    for lang, count in stats["languages"].items():
        print(f"   • {lang}: {count}")
    
    print(f"\n📁 FICHIERS GÉNÉRÉS:")
    print(f"   ✅ CSV: {os.path.basename(csv_file)}")
    print(f"   ✅ JSON: {os.path.basename(json_file)}")
    
    print(f"\n🏆 MISSION ACCOMPLIE !")
    print(f" {stats['total_entries']} entrées de thésaurus")

# Export et résumé final
if final_thesaurus:
    csv_file, json_file, statistics = export_final_thesaurus(final_thesaurus)
    display_final_summary(statistics, csv_file, json_file)
else:
    print("❌ Aucun thésaurus à exporter")

### Nettoyage des doublons

In [ ]:
# Lire le CSV (remplace 'ton_fichier.csv' par le tien)
df = pd.read_csv(csv_file, sep=';', dtype=str).fillna('')

# Fonction pour concaténer les valeurs uniques (séparées par "|")
def concat_unique(series):
    uniques = set([v.strip() for v in series if v.strip() != ''])
    return " | ".join(sorted(uniques)) if uniques else ''

# Grouper par 'ID', en concaténant les valeurs différentes pour chaque colonne
df_clean = df.groupby('ID', as_index=False).agg(concat_unique)

# Sauvegarder le résultat
df_clean.to_csv(csv_file, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ CSV nettoyé et exporté sous {csv_file}")

